In [ ]:
!pip install pretty_midi
!pip install tensorflow
!pip install midi2audio
!pip install pydub


from google.colab import files
import zipfile
import os
import numpy as np
import pretty_midi
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from midi2audio import FluidSynth
from pydub import AudioSegment





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=a882be2075bc6bc2a22dccf29263d0bb66d60adec136cd2cca109dd1f9f64133
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
# Constants
midi_directory = '/content/kpop-MIDIfiles'
sequence_length = 100
model_path = '/content/trained_model.h5'
TEMPO = 150
VELOCITY = 120
DURATION = 0.25

# Create directory for files
os.makedirs(midi_directory, exist_ok=True)

# Upload and extract files
uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(midi_directory)
    else:
        os.rename(filename, os.path.join(midi_directory, filename))

# Install required packages
!pip install pretty_midi tensorflow midi2audio pydub
!apt-get install -y fluidsynth
!wget https://github.com/FluidSynth/fluidsynth/blob/main/examples/midi/FluidR3_GM.sf2 -O FluidR3_GM.sf2

# Function to convert MIDI to note sequences
def midi_to_note_sequences(midi_file_path):
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    note_sequences = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            notes = [(note.pitch, note.start, note.end) for note in instrument.notes]
            note_sequences.append(notes)
    return note_sequences

# Function to convert notes to a fixed-length sequence
def notes_to_sequence(notes, sequence_length=sequence_length):
    sequence = np.zeros(sequence_length)
    for i, (pitch, _, _) in enumerate(notes):
        if i >= sequence_length:
            break
        sequence[i] = pitch
    return sequence

# Function to create a dataset from MIDI files
def create_dataset(midi_directory, sequence_length=sequence_length):
    sequences = []
    files = [f for f in os.listdir(midi_directory) if f.endswith('.mid')]
    for file_name in files[:20]:
        midi_file_path = os.path.join(midi_directory, file_name)
        note_sequences = midi_to_note_sequences(midi_file_path)
        for notes in note_sequences:
            sequence = notes_to_sequence(notes, sequence_length)
            sequences.append(sequence)
    return np.array(sequences)

# Function to prepare data for training
def prepare_data(sequences, sequence_length):
    X = sequences[:, :-1]
    y = sequences[:, 1:]
    y = to_categorical(y, num_classes=128)
    X = np.expand_dims(X, axis=-1)
    return X, y

# Function to build and train the LSTM model
def train_model(X, y):
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(X, y, epochs=100, batch_size=64)
    model.save(model_path)
    return model

# Function to generate new music
def generate_music(model, seed_sequence, length=200):
    generated_sequence = list(seed_sequence)
    for _ in range(length):
        x_input = np.array([generated_sequence[-sequence_length:]])
        prediction = model.predict(x_input, verbose=0)
        next_note = np.argmax(prediction[0, -1, :])
        generated_sequence.append(next_note)
    return generated_sequence

# Function to convert sequence back to MIDI format
def sequence_to_midi(sequence, output_file='generated_kpop_music.mid'):
    midi_data = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    start_time = 0
    for pitch in sequence:
        note = pretty_midi.Note(velocity=VELOCITY, pitch=int(pitch), start=start_time, end=start_time + DURATION)
        instrument.notes.append(note)
        start_time += DURATION
    midi_data.instruments.append(instrument)
    midi_data.write(output_file)

# Function to convert MIDI to WAV
def convert_midi_to_wav(midi_file, wav_file):
    fs = FluidSynth("FluidR3_GM.sf2")
    fs.midi_to_audio(midi_file, wav_file)

# Function to convert WAV to MP3
def convert_wav_to_mp3(wav_file, mp3_file):
    sound = AudioSegment.from_wav(wav_file)
    sound.export(mp3_file, format='mp3')

# Main script
if __name__ == "__main__":
    if os.path.exists(model_path):
        model = load_model(model_path)
    else:
        sequences = create_dataset(midi_directory, sequence_length)
        X, y = prepare_data(sequences, sequence_length)
        model = train_model(X, y)

    seed_sequence = sequences[0]
    generated_sequence = generate_music(model, seed_sequence, length=300)
    sequence_to_midi(generated_sequence, output_file='generated_kpop_music.mid')

    wav_file = 'generated_kpop_music.wav'
    convert_midi_to_wav('generated_kpop_music.mid', wav_file)

    mp3_file = 'generated_kpop_music.mp3'
    convert_wav_to_mp3(wav_file, mp3_file)

    files.download(mp3_file)
    print("Generated K-pop energetic music saved to 'generated_kpop_music.mp3'")

Saving BLACKPINK - _Kill This Love.mid to BLACKPINK - _Kill This Love.mid
Saving BLACKPINK - _Lovesick Girls.mid to BLACKPINK - _Lovesick Girls.mid
Saving BLACKPINK - Pink Venom MV.mp3.mid to BLACKPINK - Pink Venom MV.mp3.mid
Saving BLACKPINK - 뚜두뚜두 (DDU-DU DDU-DU) MV.mp3.mid to BLACKPINK - 뚜두뚜두 (DDU-DU DDU-DU) MV.mp3.mid
Saving BTS (방탄소년단) _달려라 방탄 (Run BTS).mid to BTS (방탄소년단) _달려라 방탄 (Run BTS).mid
Saving AnyConv.com__BLACKPINK - 'How You Like That' MV.midi to AnyConv.com__BLACKPINK - 'How You Like That' MV.midi
Saving BLACKPINK - _붐바야 (BOOMBAYAH).mid to BLACKPINK - _붐바야 (BOOMBAYAH).mid
Saving BLACKPINK - Pink Venom SPECIAL STAGE.mp3.mid to BLACKPINK - Pink Venom SPECIAL STAGE.mp3.mid
Saving JENNIE - _SOLO.mid to JENNIE - _SOLO.mid
Saving ROSÉ - _On The Ground.mid to ROSÉ - _On The Ground.mid
Saving AnyConv.com__LISA - 'MONEY' EXCLUSIVE PERFORMANCE VIDEO.midi to AnyConv.com__LISA - 'MONEY' EXCLUSIVE PERFORMANCE VIDEO.midi
Saving AnyConv.com__LISA - 'LALISA' MV.midi to AnyConv.com__LISA

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 4.9002
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step - loss: 4.6579
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step - loss: 4.4884
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step - loss: 4.3617
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step - loss: 4.2113
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step - loss: 3.9337
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 601ms/step - loss: 3.7035
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step - loss: 3.6291
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 620ms/step - loss: 3.5864
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step - loss: 3.5539
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step - loss: 3.5362
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step - loss: 3.5224
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step - loss: 3.4929
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step - loss: 3.4765
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 735ms/step - loss: 3.4683
Epoch 1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generated K-pop energetic music saved to 'generated_kpop_music.mp3'
